In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk, Dataset
import torch

/Users/jerry/workspace/book-repo/02-fine-tuning/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS device.")
else:
    device = torch.device("cpu")
    print("MPS not available, using CPU.")

Using MPS device.


In [4]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

In [5]:
dataset = load_from_disk("./data/llm_mail_dataset")
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 335
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 42
    })
    test: Dataset({
        features: ['items'],
        num_rows: 43
    })
})

# Train

In [12]:
from peft import LoraConfig

lora_config = LoraConfig(
  r=4,
  lora_alpha=8,
  lora_dropout=0,
  bias="none",
  task_type="CAUSAL_LM",
  target_modules = ["q_proj", 
    "k_proj", "v_proj", "o_proj", 
    "gate_proj", "up_proj", "down_proj"],
  modules_to_save=["lm_head"]
)

In [13]:
from peft import get_peft_model

peft_model = get_peft_model(
	model,
	lora_config,
)

/Users/jerry/workspace/book-repo/02-fine-tuning/lib/python3.13/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/Users/jerry/workspace/book-repo/02-fine-tuning/lib/python3.13/site-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [14]:
from trl import SFTTrainer, SFTConfig

In [15]:
training_args = SFTConfig(
   output_dir="trainer_output/my_mail_classifier_llm",
   report_to="none",
   overwrite_output_dir=True,
   do_train=True,
   learning_rate=2e-4,
   num_train_epochs=3,
   bf16=False,  # Disable bf16 to avoid compatibility issues
   fp16=False,  # Disable fp16 for MPS compatibility
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


In [16]:
trainer = SFTTrainer(
   model=peft_model,
   args=training_args,
   train_dataset=dataset["train"],
   eval_dataset=dataset["validation"],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

Step,Training Loss
10,2.112400
20,1.067600
30,0.949300
40,0.911800
50,0.626500
60,0.523100
70,0.525800
80,0.547300
90,0.428900
100,0.355600


TrainOutput(global_step=126, training_loss=0.7154572975067865, metrics={'train_runtime': 16003.9889, 'train_samples_per_second': 0.063, 'train_steps_per_second': 0.008, 'total_flos': 472164060015360.0, 'train_loss': 0.7154572975067865})

In [18]:
model_path = "models/my_llm_mail_classifier"
trainer.save_model(model_path)